### An example notebook on plotting up variables from the NAARC simulations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker 
import matplotlib.path as mpath
import cartopy
import cartopy.crs as ccrs
import matplotlib.patches as mpatches
import xarray as xr

import sys
import os 

import matplotlib as mpl
_string_types = (str, np.str_)

from netCDF4 import Dataset
from matplotlib.path import Path

import glob
import datetime
import pandas as pd

from matplotlib.colors import ListedColormap
import matplotlib.colors as cls

from matplotlib import rcParams

afm_filename = os.path.join(mpl.get_data_path(), 'fonts', 'afm', 'ptmr8a.afm')
from matplotlib.afm import AFM
afm = AFM(open(afm_filename, "rb"))

### Functions for plotting the data

In [ ]:
def ctitle(message,ang,dist,fig):
    ax_c = fig.add_subplot(projection="polar");
    Mbbox=afm.get_str_bbox(message)
    h0 = Mbbox[3]+Mbbox[1] # height from the 0 line
    w0 = Mbbox[2]/2
    bbox = ax_c.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    width, height = bbox.width, bbox.height
    width *= fig.dpi
    height *= fig.dpi
    sf = dist*np.pi*2*0.75 # 0.75 pt to px @ 96dpi
    fs = 10/1000
    ang += w0*fs/sf
    
    for i in range(len(message)):
    
            # Instead of a constant width, we check the width of each character.
            currentChar = message[i]
            w = afm.get_width_char(currentChar)
            w = afm.get_bbox_char(currentChar)

            if w[3]>0:
                h=w[1]/w[3]
            else:
                h=0
            c=w[0]

        
            if w[2]==0:
                w[2] = afm.get_width_char(currentChar)

            if i>0:
                os = afm.get_kern_dist(message[i-1],currentChar)
            else:
                os=0
            vos = (h0-(w[3]-w[1]))/h0
            ang -= (w[2]/2)*fs/sf
            ang -= os*fs/sf #+ c*fs/sf
            afm.get_kern_dist('H','e')
            pi_ang = ang/180*np.pi + np.pi/2
            b=ax_c.text(pi_ang, dist, currentChar, ha='center', va='center', rotation=ang, rotation_mode='anchor');

            
            ang -= (w[2]/2)*fs/sf 
    ax_c.axis('off')
    
def tcmap(rgbin, N=256):
    '''Input an array of rgb values to generate a colormap.

    :param rgbin: An [mx3] array, where m is the number of input color triplets which
         are interpolated between to make the colormap that is returned. hex values
         can be input instead, as [mx1] in single quotes with a #.
    :param N=10: The number of levels to be interpolated to.

    '''

    # rgb inputs here
    if not isinstance(rgbin[0], _string_types):
        # normalize to be out of 1 if out of 256 instead
        if rgbin.max() > 1:
            rgbin = rgbin/256.

    cmap = mpl.colors.LinearSegmentedColormap.from_list('mycmap', rgbin, N=N)

    return cmap
    
def greg_length(YR):
    # Returns length of given year in days (Gregorian calendar)
    if (YR % 400 == 0):
        LY = True 
    elif (YR % 100 == 0):
        LY = False
    elif (YR % 4 == 0):
        LY = True 
    else:
        LY = False

    if LY:
        ndays = np.array([31,29,31,30,31,30,31,31,30,31,30,31])
    else:
        ndays = np.array([31,28,31,30,31,30,31,31,30,31,30,31])

    return LY,ndays

def dmonth(doy,ndays):
    dmon = ndays*0
    for i in range(len(ndays)):
        if (doy <= ndays[i]):
            dmon[i] = doy
            break
        else:
            dmon[i] = ndays[i]
            doy = doy - ndays[i]
            if (doy < 0):
                break
    dfrac = (dmon/ndays)
    
    return dmon,dfrac


def tclock(ax,doy,YR):


    LY,ndays = greg_length(YR)
    dmon,dfrac = dmonth(doy,ndays)

    df = dfrac*3000
    N = 12
    bottom1 = 1000
    bottom2 = 5000

    theta, width = np.linspace(np.pi/12, 2 * np.pi + np.pi/12, N, endpoint=False, retstep=True)

    # Apply the scale transform and then the map coordinate transform
    #plate_carree_transform = (ccrs.PlateCarree()._as_mpl_transform(ax))
    
    ax = ax

    bars = ax.bar(
        theta, df,
        width=width-0.06,
        bottom=bottom2, 
        color="#f39c12"#, edgecolor="black"
    )
    bars = ax.bar(
        theta, [3000]*12,
        width=width-0.06,
        bottom=bottom2,
        color="#f39c12", alpha=0.2, 
    )

    bars = ax.bar(
        theta, [3000]*12,
        width=width,
        bottom=bottom1,
        color="#f39c12", alpha=0.2, 
    )

    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    ax.grid(False)
    ax.spines['polar'].set_visible(False)
    ax.set_rticks([])

    ticks = ["J","F","M","A","M","J","J","A","S","O","N","D"]
    ax.set_xticks(theta,ticks,fontsize=13)
    
    ax.text(0.5, 0.5, YR,
            horizontalalignment='center',
            verticalalignment='center',
            transform=ax.transAxes,
            fontsize=15
           );

    _ = ax

### Load colormaps for the plot

In [ ]:
rgb_speed = np.loadtxt(os.path.join('/work/n01/n01/jdha/tmp/NAARC/nemo/cfgs/NAARC/EXP_TEST/OUTPUTS_lap', 'speed.txt'))
cmap_speed = tcmap(rgb_speed, N=256)
cmap_speed_rt = tcmap(rgb_speed[::-1, :], N=256)
rgb_ice = np.loadtxt(os.path.join('/work/n01/n01/jdha/tmp/NAARC/nemo/cfgs/NAARC/EXP_TEST/OUTPUTS_lap', 'ice.txt'))
cmap_ice = tcmap(rgb_ice, N=256)
cmap_ice_rt = tcmap(rgb_ice[::-1, :], N=256)

In [ ]:
# setting up arced colorbar cmaps
chexsw=list()
for i in range(256):
    chexsw.append(cls.rgb2hex(rgb_speed[i]))
chexne=list()
for i in range(256):
    chexne.append(cls.rgb2hex(rgb_ice[i]))

### Open Data Files

In [ ]:
# Open datasets for required year
year = 1990
nc_mask = xr.open_dataset('/work/n01/n01/jdha/SN_115_4_2_1/nemo/cfgs/se-eORCA12/EXP_CANARI/INPUTS/bdy_msk.nc')
mask = nc_mask.bdy_msk
ncu = xr.open_dataset('/work/n01/n01/jdha/tmp/NAARC/nemo/cfgs/NAARC/EXP_TEST/OUTPUTS_lap/NAARC_1d_'+str(year)+'0101_'+str(year)+'1231_grid_U.nc')
ncv = xr.open_dataset('/work/n01/n01/jdha/tmp/NAARC/nemo/cfgs/NAARC/EXP_TEST/OUTPUTS_lap/NAARC_1d_'+str(year)+'0101_'+str(year)+'1231_grid_V.nc')
nci = xr.open_dataset('/work/n01/n01/jdha/tmp/NAARC/nemo/cfgs/NAARC/EXP_TEST/OUTPUTS_lap/NAARC_1d_'+str(year)+'0101_'+str(year)+'1231_icemod.nc')
nct = xr.open_dataset('/work/n01/n01/jdha/tmp/NAARC/nemo/cfgs/NAARC/EXP_TEST/OUTPUTS_lap/NAARC_1d_'+str(year)+'0101_'+str(year)+'1231_grid_T.nc')
lon = nct.nav_lon
lat = nct.nav_lat

In [ ]:
# Location of the colorbars

sw = True
nw = False
ne = True
se = False

# Setup figure
proj = ccrs.Orthographic(central_longitude=-20.0, central_latitude=45.0)

cmsize = [25,25]   
fig = plt.figure(figsize=[x/2.54 for x in cmsize], dpi=96)

#for time in range(365):
for time in [364,]: # For this example just pull out one time slice
    
    # Derive surface currents
    current = xr.zeros_like(ncu.sozocrtx[0,:,:].squeeze())
    current[1:,1:] = ( ((ncu.sozocrtx[time,1:,:-1]+ncu.sozocrtx[time,1:,1:])*0.5)**2 + ((ncv.somecrty[time,:-1,1:]+ncv.somecrty[time,1:,1:])*0.5)**2) ** 0.5
    current[1:,0] = ( ((ncu.sozocrtx[time,1:,0]+ncu.sozocrtx[time,1:,-1])*0.5)**2 + ((ncv.somecrty[time,:-1,0]+ncv.somecrty[time,1:,0])*0.5)**2) ** 0.5
    current = xr.where(mask==0, np.nan, np.log(current))
    
    # Overlay ice speed
    ices = nci.sivelo[time,:,:]
    ices = xr.where(mask==0, np.nan, ices)
    ices = np.where(ice==0,np.nan,ices)
    ices = np.where(ice>1,np.nan,np.log(ices))
    
    ax = fig.add_subplot(1,1,1, projection=proj)


    if sw:
        
        values = np.array([0, 0.05, 0.08, 0.14, 0.22, 0.37, 0.61, 1.  ])
        ax_sw = fig.add_subplot(projection="polar");
    
        ax_sw.bar(x=np.linspace(np.pi,np.pi/2*3,256), width=0.01, height=0.3, bottom=3,
               color=chexsw, align="edge");
        
        for loc, val in zip(np.arange(np.pi,np.pi/2*3+np.pi/7.,np.pi/2/7.).tolist(), values.tolist()):
            plt.annotate(val, xy=(loc, 3.5), rotation=loc*(180/np.pi)+180, ha="center", va="center",annotation_clip=False);
        
        ax_sw.axis('off')
        ctitle('Surface Current (m/s)',135,1.12,fig)
        
    if ne:
        
        values = np.round(np.arange(0,1+1./10.,1./10.)*10)/10
        values = np.array([0, 0.05, 0.08, 0.14, 0.22, 0.37, 0.61, 1.  ])
        ax_ne = fig.add_subplot(projection="polar");
        
        ax_ne.bar(x=np.linspace(0,np.pi/2,256), width=0.01, height=0.3, bottom=3,
               color=chexne, align="edge");
        
        for loc, val in zip(np.arange(0,np.pi/2+np.pi/7.,np.pi/2/7.).tolist(), values.tolist()):
            plt.annotate(val, xy=(loc, 3.5), rotation=loc*(180/np.pi), ha="center", va="center",annotation_clip=False);
        
        ax_ne.axis('off')
        ctitle('Ice Speed (m/s)',-45,1.12,fig)


    ax.set_global()
    ax.stock_img()
    ax.coastlines("50m", linewidth=0.5, color="grey")
    if 1==1:
        ht=ax.pcolormesh(lon, lat, current, 
                          cmap=cmap_speed,
                          transform=ccrs.PlateCarree(),
                          shading='auto', vmin=-3, vmax=0)
    
        hi=ax.pcolormesh(lon, lat, ices, 
                          cmap=ice_cm,
                          transform=ccrs.PlateCarree(),
                          shading='auto', vmin=-3, vmax=0)

    axin1 = ax.inset_axes([0.60, 0.25, 0.14, 0.14], polar=True)    
    tclock(axin1,time+1,year)      
    time_str = (f"{time+1:03d}")

    fig.tight_layout() 
    a = ax.get_position().bounds
    
    perc=0.88
    ax.set_position([a[0]+(a[2]-a[2]*perc)/2, a[1]+(a[3]-a[3]*perc)/2, a[2]*perc, a[3]*perc])

    #plt.savefig('/work/n01/n01/jdha/tmp/NAARC/nemo/cfgs/NAARC/EXP_TEST/OUTPUTS/speed_'+time_str)
    #plt.clf()

    plt.show()